<a href="https://colab.research.google.com/github/SF1308/LLMS-training/blob/main/Detecci%C3%B3n_de_correos_maliciosos_con_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ensure the latest OpenAI library version is installed
!pip install --upgrade openai

 #1 Configuración de la API Key de OpenAI

In [ ]:
# Step 1. Configuring the OpenAI API Key

import os

# Load the API key from a secure location, such as Google Drive, and set it in the environment
with open("{{your_openAI_api_key}}") as f:
    os.environ["OPENAI_API_KEY"] = f.readline().strip()

In [ ]:
print(os.environ["OPENAI_API_KEY"])

# 2. Email classification function with ICL

In [ ]:
import os
from openai import OpenAI

# Initialize the OpenAI client with the API key
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def classify_email_ic(email_content):
    """
    Classifies an email as 'Malicious' or 'Benign' using a role-specific prompt with examples.

    Args:
        email_content (str): The content of the email to be classified.

    Returns:
        str: 'Malicious' or 'Benign' based on the model's evaluation.
    """
    # Prepare the messages for ChatCompletion
    messages = [
        {"role": "system", "content": "You are a cybersecurity expert specialized in identifying malicious emails, such as phishing or spam. Analyze the content of each email and classify it as 'Malicious' or 'Benign' based on suspicious patterns."},
        {"role": "user", "content": """
Examples:
---
Email: "Dear customer, your bank account has been blocked. Click on the following link to verify your details: http://fake-link.com"
Classification: Malicious
---
Email: "Hi John, would you like to go out for dinner tonight? Let me know! - Sarah"
Classification: Benign
---
Email: "Congratulations, you won a cash prize. Click here to claim: http://fake-prize.com"
Classification: Malicious
---
Email: "Dear user, your subscription to our service will expire soon. Visit our site for more information: https://legit-company.com/update"
Classification: Benign
---
Email: "ATTENTION! We detected an issue with your account. Verify immediately by clicking here: http://unsafe-link.com"
Classification: Malicious
---

Classify the following email:

Email:
""" + email_content + "\nClassification:"}
    ]

    # Create the chat completion with the new client format
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=10,
        temperature=0
    )

    # Accessing the classification result from the response using object attributes
    classification = response.choices[0].message.content.strip()  # Changed this line
    return classification

# 3. Sample email classification

In [ ]:
# 3. Sample email classification with updated Chat API
sample_email = """
Dear user, your account has been temporarily suspended.
To reactivate your account, please click on the following link: http://fake-link.com
Best regards, Support Team.
"""

# Classify the sample email
classification = classify_email_ic(sample_email)
print(f"Sample email classification: {classification}")


Sample email classification: Classification: Malicious


# 4. Automate classification of emails from the inbox using the Gmail API

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Define the scope for read-only access to Gmail
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def main():
    """Reads emails with the label 'purchases/orders' and classifies them."""
    creds = None
    # Load credentials or authenticate if necessary
    if os.path.exists("{{TOKEN.json}}"):
        creds = Credentials.from_authorized_user_file("{{TOKEN.json}}", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "{{your_credentials.json}}", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for future use
        with open("{{TOKEN.json}}", "w") as token:
            token.write(creds.to_json())

    try:
        # Build the Gmail API service
        service = build("gmail", "v1", credentials=creds)

        # Search for emails with the label 'purchases/orders'
        results = service.users().messages().list(userId="me", labelIds=["Label_2632497382483817671"]).execute()
        messages = results.get("messages", [])

        if not messages:
            print("No emails found with the label 'purchases/orders'.")
            return

        print("Emails with label 'purchases/orders':")
        for message in messages:
            msg = service.users().messages().get(userId="me", id=message["id"]).execute()
            email_content = msg.get("snippet", "No snippet available")

            # Classify the email content using classify_email_ic function
            classification = classify_email_ic(email_content)
            print(f"Email ID: {message['id']}\nContent:\n{email_content}\nClassification: {classification}\n")

    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == "__main__":
    main()


Emails with label 'purchases/orders':
Email ID: 18e4e15aea35fefa
Content:
ESTIMADO CLIENTE ADJUTNO ENCONTRARA DETALLE DE LA FACTURA A ABONAR POR FAVOR ENVIE COMPROBANTE DEL APGO INDICANDO NUMERO DE FACTURA CUALQUIER CONSULTA , A DISPOSICION MYPREPCENTERLLC@GMAIL.COM
Classification: Classification: Malicious

This email exhibits suspicious characteristics

Email ID: 18e4a1cd7a597589
Content:
ESTIMADO CLIENTE ADJUTNO ENCONTRARA DETALLE DE LA FACTURA A ABONAR POR FAVOR ENVIE COMPROBANTE DEL APGO INDICANDO NUMERO DE FACTURA CUALQUIER CONSULTA , A DISPOSICION MYPREPCENTERLLC@GMAIL.COM
Classification: Classification: Malicious

This email exhibits suspicious characteristics

Email ID: 18e021d4fa9e5c37
Content:
ESTIMADO CLIENTE ADJUTNO ENCONTRARA DETALLE DE LA FACTURA A ABONAR POR FAVOR ENVIE COMPROBANTE DEL APGO INDICANDO NUMERO DE FACTURA CUALQUIER CONSULTA , A DISPOSICION MYPREPCENTERLLC@GMAIL.COM
Classification: Classification: Malicious

This email exhibits suspicious characteristics

Em